In [21]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

import tensorflow as tf 
import os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img


from tqdm import tqdm
from tensorflow.keras.utils import to_categorical 

# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import LearningRateSchedul

# from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
TrainDir = r"images/train" 
TestDir = r"images/validation"

In [23]:
def createDataFrame(Dir):
  imgPaths = []
  imgLabels = []
  
  for label in os.listdir(Dir):
    for img in os.listdir(os.path.join(Dir,label)):
      imgPaths.append(os.path.join(Dir,label,img))
      imgLabels.append(label)
    print(label, "Completed")
  return imgPaths, imgLabels

In [24]:
train = pd.DataFrame()
test = pd.DataFrame()

train['images'], train['labels'] = createDataFrame(TrainDir)
print()
test['images'], test['labels'] = createDataFrame(TestDir)

angry Completed
disgust Completed
fear Completed
happy Completed
neutral Completed
sad Completed
surprise Completed

angry Completed
disgust Completed
fear Completed
happy Completed
neutral Completed
sad Completed
surprise Completed


In [25]:
train

,images,labels
0,images/train\angry\0.jpg,angry
1,images/train\angry\1.jpg,angry
2,images/train\angry\10.jpg,angry
3,images/train\angry\10002.jpg,angry
4,images/train\angry\10016.jpg,angry
...,...,...
28816,images/train\surprise\9969.jpg,surprise
28817,images/train\surprise\9985.jpg,surprise
28818,images/train\surprise\9990.jpg,surprise
28819,images/train\surprise\9992.jpg,surprise


In [26]:
test

,images,labels
0,images/validation\angry\10052.jpg,angry
1,images/validation\angry\10065.jpg,angry
2,images/validation\angry\10079.jpg,angry
3,images/validation\angry\10095.jpg,angry
4,images/validation\angry\10121.jpg,angry
...,...,...
7061,images/validation\surprise\9806.jpg,surprise
7062,images/validation\surprise\9830.jpg,surprise
7063,images/validation\surprise\9853.jpg,surprise
7064,images/validation\surprise\9878.jpg,surprise


In [27]:
def featureExtraction(images):
    imgFeatures = []
    
    for image in tqdm(images):
        img = load_img(image,color_mode='grayscale')
        img = np.array(img)
        imgFeatures.append(img)
    imgFeatures = np.array(imgFeatures)
    imgFeatures = imgFeatures.reshape(len(imgFeatures),48,48,1)
    
    return imgFeatures

In [28]:
train_features = featureExtraction(train['images'])

100%|██████████| 28821/28821 [00:03<00:00, 7842.54it/s]


In [29]:
test_features = featureExtraction(test['images'])

100%|██████████| 7066/7066 [00:00<00:00, 7678.41it/s]


In [30]:
x_train = train_features/255.0
x_test = test_features/255.0


In [31]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
le.fit(train['labels'])

y_train = le.transform(train['labels'])
y_test = le.transform(test['labels'])

In [32]:
y_train = to_categorical(y_train,num_classes=7)
y_test = to_categorical(y_test,num_classes=7)

In [34]:
# Import necessary libraries
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Define paths for your training and validation directories
train_dir = 'images/train'  # Replace with your training data directory
val_dir = 'images/validation'      # Replace with your validation data directory

# Create ImageDataGenerators for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize image pixels to [0, 1]
    rotation_range=20,       # Randomly rotate images in the range (degrees)
    width_shift_range=0.2,   # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,         # Randomly shear images
    zoom_range=0.2,          # Randomly zoom images
    horizontal_flip=True,    # Randomly flip images horizontally
    fill_mode='nearest'      # Strategy to fill in newly created pixels
)

val_datagen = ImageDataGenerator(rescale=1./255)  # Validation data should not be augmented

# Set up data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,               # Path to training data directory
    target_size=(224, 224),  # Resize images to fit the model input size
    batch_size=32,
    class_mode='categorical' # For multi-class classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,                 # Path to validation data directory
    target_size=(224, 224),  # Resize images
    batch_size=32,
    class_mode='categorical' # For multi-class classification
)

# Model setup - Transfer learning with ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of ResNet50
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # Output layer for multi-class classification
])

# Compile the model
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Train the model
model.fit(
    train_generator, 
    epochs=100,
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint],
    batch_size=32
)

# Unfreeze layers of ResNet50 for fine-tuning after some epochs (optional)
for layer in base_model.layers:
    layer.trainable = True

# Recompile the model after unfreezing the layers
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])


# Continue training with fine-tuning
model.fit(
    train_generator, 
    epochs=50,  # You can adjust the number of epochs for fine-tuning
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint],
    batch_size=32
)


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


ValueError: Argument(s) not recognized: {'lr': 0.0001}

In [ ]:


model_json = model.to_json()
with open("emothionDetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emothionDetector.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file = open("emothionDetector.json", 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emothionDetector.h5")


label = ['angry','disgust','fear','happy','neutral','sad','surprise']

def ef(image):
    img = load_img(image,color_mode='grayscale')
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0


image = 'images/train/fear/49.jpg'
print("Original Image is of fear")


img = ef(image)

pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model predictoin is : ", pred_label)



plt.imshow(img.reshape(48,48), cmap='gray')



Original Image is of fear
1/1 [==============================] - 0s 56ms/step
model predictoin is :  fear
